# Chapter 6: Precognition (Thinking Step by Step)

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
# !pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.environ.get("ANTHROPIC_API_KEY")
MODEL_NAME = "claude-3-haiku-20240307"

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Lesson

If someone woke you up and immediately started asking you several complicated questions that you had to respond to right away, how would you do? Probably not as good as if you were given time to **think through your answer first**. 

Guess what? Claude is the same way.

**Giving Claude time to think step by step sometimes makes Claude more accurate**, particularly for complex tasks. However, **thinking only counts when it's out loud**. You cannot ask Claude to think but output only the answer - in this case, no thinking has actually occurred.

### Examples

In the prompt below, it's clear to a human reader that the second sentence belies the first. But **Claude takes the word "unrelated" too literally**.

In [2]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

The sentiment of this movie review is positive.

The review states that the movie "blew my mind with its freshness and originality", which indicates a very positive and enthusiastic reaction to the film.

The second part of the review about living under a rock since 1900 is likely meant as a humorous or self-deprecating remark, but it does not negate the overall positive sentiment expressed about the movie's qualities.


To improve Claude's response, let's **allow Claude to think things out first before answering**. We do that by literally spelling out the steps that Claude should take in order to process and think through its task. Along with a dash of role prompting, this empowers Claude to understand the review more deeply.

In [3]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

<positive-argument>
- The review suggests the movie has a sense of freshness and originality, which can be seen as positive attributes.
- The reviewer seems genuinely impressed and enthusiastic about the movie, indicating they found it to be a compelling and mind-blowing experience.
</positive-argument>

<negative-argument>
- The reviewer's comment about living under a rock since 1900 implies that the movie's "freshness and originality" may not be as novel or groundbreaking as the reviewer suggests, as the reviewer appears to be out of touch with current trends and developments in the film industry.
- The sarcastic tone of the last sentence could be interpreted as a negative or dismissive attitude towards the movie, undermining the positive sentiment expressed earlier.
</negative-argument>

Based on the arguments presented, the overall sentiment of the review appears to be mixed. While the reviewer initially expresses positive sentiments about the movie's "freshness and originality," t

**Claude is sometimes sensitive to ordering**. This example is on the frontier of Claude's ability to understand nuanced text, and when we swap the order of the arguments from the previous example so that negative is first and positive is second, this changes Claude's overall assessment to positive.

In most situations (but not all, confusingly enough), **Claude is more likely to choose the second of two options**, possibly because in its training data from the web, second options were more likely to be correct.

In [4]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

<negative-argument>
The reviewer's statement about living under a rock since 1900 could be interpreted as a negative comment, suggesting that the movie's "freshness and originality" are not actually new or innovative, but rather something the reviewer has been unaware of for a long time.
</negative-argument>

<positive-argument>
The reviewer's praise for the movie's "freshness and originality" is a clear positive sentiment, indicating that the reviewer found the movie to be innovative and engaging. The fact that the reviewer has been "living under a rock" could be seen as a humorous way of acknowledging their own lack of exposure to recent developments, rather than a criticism of the movie itself.
</positive-argument>

Based on the overall tone and content of the review, the sentiment is more positive than negative. The reviewer's praise for the movie's "freshness and originality" outweighs the potential negative implication of the "living under a rock" comment.


**Letting Claude think can shift Claude's answer from incorrect to correct**. It's that simple in many cases where Claude makes mistakes!

Let's go through an example where Claude's answer is incorrect to see how asking Claude to think can fix that.

In [5]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

Here's a famous movie starring an actor born in 1956:

The Shawshank Redemption (1994) starring Tim Robbins. Tim Robbins was born on October 16, 1958.


Let's fix this by asking Claude to think step by step, this time in `<brainstorm>` tags.

In [6]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))

Here's a brainstorm of some actors and their birth years:

<brainstorm>
- Tom Hanks (1956)
- Denzel Washington (1954)
- Julia Roberts (1967)
- Harrison Ford (1942)
- Meryl Streep (1949)
</brainstorm>

A famous movie starring an actor born in 1956 is:

Forrest Gump, starring Tom Hanks.


If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 6.1 - Classifying Emails](#exercise-61---classifying-emails)
- [Exercise 6.2 - Email Classification Formatting](#exercise-62---email-classification-formatting)

### Exercise 6.1 - Classifying Emails
In this exercise, we'll be instructing Claude to sort emails into the following categories:										
- (A) Pre-sale question
- (B) Broken or defective item
- (C) Billing question
- (D) Other (please explain)

For the first part of the exercise, change the `PROMPT` to **make Claude output the correct classification and ONLY the classification**. Your answer needs to **include the letter (A - D) of the correct choice, with the parentheses, as well as the name of the category**.

Refer to the comments beside each email in the `EMAILS` list to know which category that email should be classified under.

In [11]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as one of four categories <category>(A) Pre-sale question, (B) Broken or defective item, (C) Billing question, (D) Other (please explain)</category>, the answer must include both category (letter) and its name : {email}"""

# Prefill for Claude's response, if any
PREFILL = "("

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

<>:25: SyntaxWarning: invalid escape sequence '\)'
<>:26: SyntaxWarning: invalid escape sequence '\)'
<>:27: SyntaxWarning: invalid escape sequence '\)'
<>:28: SyntaxWarning: invalid escape sequence '\)'
<>:25: SyntaxWarning: invalid escape sequence '\)'
<>:26: SyntaxWarning: invalid escape sequence '\)'
<>:27: SyntaxWarning: invalid escape sequence '\)'
<>:28: SyntaxWarning: invalid escape sequence '\)'
/var/folders/n7/xhlh4rqs7fb3rcyrfgmr1vyr0000gn/T/ipykernel_92149/3611586998.py:25: SyntaxWarning: invalid escape sequence '\)'
  "A": "A\) P",
/var/folders/n7/xhlh4rqs7fb3rcyrfgmr1vyr0000gn/T/ipykernel_92149/3611586998.py:26: SyntaxWarning: invalid escape sequence '\)'
  "B": "B\) B",
/var/folders/n7/xhlh4rqs7fb3rcyrfgmr1vyr0000gn/T/ipykernel_92149/3611586998.py:27: SyntaxWarning: invalid escape sequence '\)'
  "C": "C\) B",
/var/folders/n7/xhlh4rqs7fb3rcyrfgmr1vyr0000gn/T/ipykernel_92149/3611586998.py:28: SyntaxWarning: invalid escape sequence '\)'
  "D": "D\) O"


--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please classify this email as one of four categories <category>(A) Pre-sale question, (B) Broken or defective item, (C) Billing question, (D) Other (please explain)</category>, the answer must include both category (letter) and its name : Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.

ASSISTANT TURN
(

------------------------------------- Claude's response -------------------------------------
B) Broken or defective item

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True 






--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please classify this email as one of four categories <category>(A) Pre-sale question, (B) Broken or de

❓ If you want a hint, run the cell below!

In [10]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

The grading function in this exercise is looking for the correct categorization letter + the closing parentheses and the first letter of the name of the category, such as "C) B" or "B) B" etc.
Let's take this exercise step by step:										
1.	How will Claude know what categories you want to use? Tell it! Include the four categories you want directly in the prompt. Be sure to include the parenthetical letters as well for easy classification. Feel free to use XML tags to organize your prompt and make clear to Claude where the categories begin and end.									
2.	Try to cut down on superfluous text so that Claude immediately answers with the classification and ONLY the classification. There are several ways to do this, from speaking for Claude (providing anything from the beginning of the sentence to a single open parenthesis so that Claude knows you want the parenthetical letter as the first part of the answer) to telling Claude that you want the classification and only the classifica

Still stuck? Run the cell below for an example solution.						

In [12]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)


USER TURN
Please classify this email into the following categories: {email}

Do not include any extra words except the category.

<categories>
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question
(D) Other (please explain)
</categories>

ASSISTANT TURN
(



### Exercise 6.2 - Email Classification Formatting
In this exercise, we're going to refine the output of the above prompt to yield an answer formatted exactly how we want it. 

Use your favorite output formatting technique to make Claude wrap JUST the letter of the correct classification in `<answer></answer>` tags. For instance, the answer to the first email should contain the exact string `<answer>B</answer>`.

Refer to the comments beside each email in the `EMAILS` list if you forget which letter category is correct for each email.

In [14]:
# Prompt template with a placeholder for the variable content
PROMPT = """
Please classify this email into these categories, only answer with the letter inside <answer> tag
<categories>
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question
(D) Other (please explain)
</categories>
: {email}
"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN

Please classify this email into these categories, only answer with the letter inside <answer> tag
<categories>
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question
(D) Other (please explain)
</categories>
: Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.


ASSISTANT TURN


------------------------------------- Claude's response -------------------------------------
<answer>B</answer>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True 






--------------------------- Full prompt with variable substutions ---------------------------
USER TURN

Please classify this email into these categories, only answer with the letter inside <answer> tag
<categories>
(A) Pre-sal

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))